# Within-host SNVs in individuals with multiple SCAN samples

In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO

## Load files

In [77]:
def load_df(file):
    '''
    Loads metadata tsv as df.
    '''
    with open(file) as tfile:
        df = pd.read_csv(tfile, sep = '\t')
        if 'nwgc_id' in df.columns:
            df['nwgc_id'] = df.nwgc_id.astype('str')
        if 'position' and 'variant' in df.columns:
            df['snv'] = df['position'].astype('str').str.cat(df['variant'],sep="")
        df.sort_values(by=['individual_identifier', 'date'], inplace=True)
    return df

metadata = load_df('../results/metadata_scan_all.tsv')

snvs = load_df('../results/snvs_scan_all.tsv')

In [78]:
def load_genomes(file):
    '''
    Loads genomes as dictionary
    '''
    with open(file, 'r') as f:
        genomes = SeqIO.to_dict(SeqIO.parse(f, 'fasta'))
    return genomes

genomes = load_genomes('../results/aligned.fasta')


## Find individuals with multiple samples

In [84]:
def get_multiples(df):
    '''
    Returns df with just samples from multiple individuals
    '''
    multiples = df.groupby(by=['individual_identifier']).filter(lambda x: len(x) > 1).reset_index(drop=True)
    n_individ = int(len(multiples)/2)
    multiples['individual'] = [i for num in range(1, n_individ + 1) for i in [num]*2]
    return multiples

multiple_df = get_multiples(metadata)

In [85]:
multiple_df

,batch,nwgc_id,strain,avg_ct,date,location,individual_identifier,age,puma,address_identifier,symptom_onset,individual
0,20200416_fastq,388977,USA/WA-S301/2020,23.063411,2020-03-14,Pierce County,3f8fb75bfe48f31dc142c5bd629fc8d542f9a89f4973bd...,17.00,5311503.0,NaN,NaN,1
1,20200416_fastq,389358,USA/WA-S286/2020,29.704358,2020-03-15,Pierce County,3f8fb75bfe48f31dc142c5bd629fc8d542f9a89f4973bd...,17.00,5311503.0,NaN,NaN,1
2,20200831_fastq,438095,USA/WA-S2719/2020,21.992674,2020-07-02,King County,628b6616a28049b1099959c884f083a3be62e8983ba7a8...,85.00,5311613.0,b4b7055fc0b861638213a93cff8d5898b8990f18a536cb...,2020-07-01,2
3,20200909_fastq,444373,USA/WA-S2769/2020,26.053100,2020-07-13,King County,628b6616a28049b1099959c884f083a3be62e8983ba7a8...,85.00,5311613.0,b4b7055fc0b861638213a93cff8d5898b8990f18a536cb...,NaN,2
4,20200909_fastq,457563,USA/WA-S2771/2020,23.799139,2020-08-01,King County,d9149c24ce3b5bffce436c93616efce1c66848d794e1a8...,63.75,5311607.0,fb859bab379819770195f951ddf2ba45d3402656ca8eab...,2020-07-26,3
5,20200909_fastq,472208,USA/WA-S2788/2020,28.146982,2020-08-12,King County,d9149c24ce3b5bffce436c93616efce1c66848d794e1a8...,63.83,5311607.0,fb859bab379819770195f951ddf2ba45d3402656ca8eab...,NaN,3


So three individuals have multiple samples. In one person, samples are separated by 1 day. In two people, samples are separated by 11 days. All persons have a "low (<24) Ct" sample and a higher Ct sample. I'm not convinced that variants can be reliably called from high Ct samples, but this might be an unavoidable challenge of having samples from across an infection time course.


## Comparing consensus genomes from each person's samples

In [87]:
def convert(df, genomes):
    '''
    Checks if consensus genomes in samples from same people are identical to each other.
    '''
    mapping = {}
    for person in df.individual.unique():
        strains = df.loc[df.individual==person, 'strain']
        for strain in strains:
            seq = list(genomes[strain].seq)
            seq_conversion = {
                    'C': 2,
                    'T': 3,
                    'G': 5,
                    'A': 7,
                    'D': 105,
                    'R': 35,
                    'Y': 6,
                    'H': 42,
                    'K': 15,
                    'V': 70,
                    'N': 210,
                    'M': 14,
                    'W': 21,
                    'S': 10,
                    '-': 210
            }
            sequence = [seq_conversion.get(base) for base in seq]
            array = np.asarray(sequence, dtype=int)
            mapping[strain] = array
    return mapping

multiple_genomes = convert(multiples_df, genomes)

def compare(array1, array2):
    '''
    Identifies SNPs between sequences where sequences are represented as numpy integer arrays.
    CTGA must be represented by prime numbers and ambiguous bases are the multiples of those prime numbers.
    '''
    comparison = {}
    condition1 = (array1 != array2)
    condition2 = (array1 % array2 != 0)
    condition3 = (array2 % array1 != 0)
    location = np.where(condition1 & condition2 & condition3)
    position = [pos + 1 for pos in location] # Plus one makes sure that bp numbering begins at 1, not zero
    base1 = array1[location]
    base2 = array2[location]
    seq_conversion = {
        2:'C',
        3:'T',
        5:'G',
        7:'A',
        105:'D',
        35:'R',
        6:'Y',
        42:'H',
        15:'K',
        70:'V',
        210:'N',
        14:'M',
        21:'W',
        10:'S'
    }
    comparison['location'] = position
    comparison['base1'] = [seq_conversion.get(base) for base in base1]
    comparison['base2'] = [seq_conversion.get(base) for base in base2]
    return comparison

def compare_pairwise(df, mapping):
    '''
    Compares all strains pairwise storing the genetic distance between each strains as NxN arrays at the genome & segment level.
    '''
    pairwise = {}
    for person in df.individual.unique():
        strains = df.loc[df.individual==person, 'strain'].reset_index(drop=True)
        pairwise[person] = compare(mapping[strains[0]], mapping[strains[1]])
    return pairwise

compare_pairwise(multiple_df, multiple_genomes)

{1: {'location': [array([], dtype=int64)], 'base1': [], 'base2': []},
 2: {'location': [array([27689])], 'base1': ['T'], 'base2': ['C']},
 3: {'location': [array([], dtype=int64)], 'base1': [], 'base2': []}}

So consensus genomes are identical for persons 1 & 3. Consensus genome has one substitution for person 2: T27689C.


## Comparing iSNVs from each person's samples

In [88]:
def filter_snvs(df, multiples):
    '''
    Filters snvs dataframe to individuals with multiple samples
    '''
    ids = multiples['nwgc_id']
    filtered = df[df.nwgc_id.isin(ids)].reset_index(drop=True)
    merged = filtered.merge(multiples, how='left')
    return merged

multiple_snvs = filter_snvs(snvs, multiples_df)

In [93]:
def n_snvs(variants, metadata):
    for person in metadata.individual.unique():
        print('Person: ' + str(person))
        for strain in metadata.loc[metadata.individual==person, 'nwgc_id']:
            n_samples = len(variants[variants.nwgc_id==strain])
            print(' # of iSNVs in sample: ' + str(n_samples))

n_snvs(multiple_snvs, multiple_df)

Person: 1
 # of iSNVs in sample: 22
 # of iSNVs in sample: 58
Person: 2
 # of iSNVs in sample: 1
 # of iSNVs in sample: 69
Person: 3
 # of iSNVs in sample: 1
 # of iSNVs in sample: 22


Note that all the highr Ct (later timepoint) samples have more variants. 

In [96]:
def compare_snvs(df):
    '''
    Adds whether or not a SNV in one sample is shared with the other sample from that person.
    '''
    counter = 0
    for person in df.individual.unique():
        person_df = df[df.individual==person].copy()       
        shared_snvs = person_df.groupby('snv').filter(lambda x: len(x) > 1)
        snvs = shared_snvs['snv'].unique()
        status = []
        for i in person_df.index:
            if person_df.loc[i, 'snv'] in snvs:
                status.append('shared')
            else:
                status.append('not_shared')
        person_df['is_shared'] = status
        if counter == 0:
            joined_df = person_df
            counter = 1
        else:
            joined_df = joined_df.append(person_df)
    return joined_df

multiple_snvs = compare_snvs(multiple_snvs)

multiple_snvs[multiple_snvs.is_shared=='shared']
        

,strain,nwgc_id,position,variant,genome,protein,aa_position,aa_variant,aa_genome,mut_type,...,frequency,symptom_onset,batch,age,date,avg_ct,address_identifier,snv,individual,is_shared
11,USA/WA-S301/2020,388977,9502,T,C,ORF1a,3079,A,A,S,...,0.058146,NaN,20200416_fastq,17.0,2020-03-14,23.063411,NaN,9502T,1,shared
18,USA/WA-S301/2020,388977,22133,A,G,S,191,K,E,NS,...,0.013393,NaN,20200416_fastq,17.0,2020-03-14,23.063411,NaN,22133A,1,shared
30,USA/WA-S286/2020,389358,9502,T,C,ORF1a,3079,A,A,S,...,0.170732,NaN,20200416_fastq,17.0,2020-03-15,29.704358,NaN,9502T,1,shared
57,USA/WA-S286/2020,389358,22133,A,G,S,191,K,E,NS,...,0.033184,NaN,20200416_fastq,17.0,2020-03-15,29.704358,NaN,22133A,1,shared


Only person 1 has shared iSNVs: 9502T & 22133A. Frequency of both variants increases in second sample compared to the first sample. These are iSNVs that are relatively common in the SCAN dataset. I'm somewhat surprised that more iSNVs aren't shared considering the samples are only one day apart; it boosts my suspicion that we're having a hard time reliably calling variants in high Ct samples (especially w/o replicate sequencing to verify variants).

In [97]:
multiple_snvs[multiple_snvs.nwgc_id=='438095']

,strain,nwgc_id,position,variant,genome,protein,aa_position,aa_variant,aa_genome,mut_type,...,frequency,symptom_onset,batch,age,date,avg_ct,address_identifier,snv,individual,is_shared
80,USA/WA-S2719/2020,438095,27689,C,T,ORF7a,99,P,L,NS,...,0.066038,2020-07-01,20200831_fastq,85.0,2020-07-02,21.992674,b4b7055fc0b861638213a93cff8d5898b8990f18a536cb...,27689C,2,not_shared


The one iSNV in Person 2's first sample is also the same variant that was fixed in the consensus genome of the second sample. That's cool!